This script generates 4C-like contact profiles from a HiC matrix in cooler format binned at 1 kilobase. It requires a file listing the chromosomes and their length (S288c_chr_centro_coordinates.tsv provided with the reference genome file). It takes three main arguments:
- the position of the viewpoint (DSB_pos, that will be translated into anchor_bin)
- the size of the viewpoint to consider (the bin_size, on each side of the viewpoint)
- the binning of the contacts to generate the 4C plot (the bin_size_x)



#### Load the packages and set the working directories

In [1]:
import numpy as np
import sys
import pandas as pd
import datetime
import scipy
from scipy import stats
from scipy import signal
import pip
import hicstuff as hcs
import hicstuff.io as hio
import hicstuff.view as hcv
import hicstuff.digest as hcd
from Bio import SeqIO
from Bio.Seq import Seq
print(pd.__version__)
print(np.__version__)
todense = lambda x: hcv.sparse_to_dense(x, remove_diag=False)

dir_seqs= "/mnt/f/Nextcloud/Sequences/Genomes/"
input_dir = "/mnt/e/Science/HiC/Contact_files/"
out_dir = "/mnt/f/Nextcloud/DR07_UMR5239_Experiments/HiC/4C_analysis/Condensin_normalized/"

1.3.5
1.21.5


Define function for SCN normalized 4C-like at the RE on chr3 and 6 equivalent location in the genome relative to arm length and centro distance

In [10]:
def Function_RE(frags, mat, chrs, i, out_dir, max_mad, kernel_size, poly):
    #kernel = np.ones(kernel_size)/kernel_size
    VP_bin_size = 3000 #in bp
    chrom = ["chr3", "chr5", "chr8", "chr14", "chr2", "chr11", "chr13"]
    VP = [29000, 66000, 21000, 713000, 153000, 525631, 182646]
    orientation = ["F", "F", "F", "R", "F", "R", "F"]

    array_length = int(len(frags.loc[(frags["chrom"]== chrom[0])]) - (VP[0]/1000))
    
    #set the main VP
    main_VP_bin=np.array(frags.loc[(frags["chrom"]== chrom[0]) & (frags["start_pos"] >= VP[0]) & (frags["start_pos"] < (VP[0]+VP_bin_size))].index.values)
    
    #define the stretch of data to look at for the main VP
    if orientation[0] == "F":
        main_VP_values=np.array(frags.loc[(frags["chrom"]== chrom[0]) & (frags["start_pos"] >= VP[0])].index.values)
    else:
        main_VP_values=np.array(frags.loc[(frags["chrom"]== chrom[0]) & (frags["start_pos"] <= VP[0])].index.values)
        
    distance_bp = len(main_VP_values)*1000
    
    main_VP_mat = mat[min(main_VP_bin):max(main_VP_bin),main_VP_values]
    main_VP_mat = np.mean(main_VP_mat, axis=0)
    main_VP_mat = main_VP_mat/sum(main_VP_mat)
    final_mat = main_VP_mat
    #final_mat_smooth = np.convolve(final_mat, kernel, mode='same')
    final_mat_smooth = scipy.signal.savgol_filter(final_mat, kernel_size, poly)

    for j in range(1, len(chrom),1):
        ctl_VP_bin=np.array(frags.loc[(frags["chrom"]== chrom[j]) & (frags["start_pos"] >= VP[j]) & (frags["start_pos"] < (VP[j]+VP_bin_size))].index.values)
        #print(chrom[j])
        #print(VP[j])
        #print(orientation[j])
        if orientation[j] == "F":
            ctl_VP_values=np.array(frags.loc[(frags["chrom"]== chrom[j]) & (frags["start_pos"] >= VP[j]) & (frags["start_pos"] < (VP[j]+distance_bp))].index.values)
            ctl_VP_mat = mat[min(ctl_VP_bin):max(ctl_VP_bin),ctl_VP_values]
            ctl_VP_mat = np.mean(ctl_VP_mat, axis=0)
            ctl_VP_mat = ctl_VP_mat/sum(ctl_VP_mat)
           
            final_mat = np.column_stack((final_mat,ctl_VP_mat))
        else:
            ctl_VP_values=np.array(frags.loc[(frags["chrom"]== chrom[j]) & (frags["start_pos"] > (VP[j]-distance_bp)) & (frags["start_pos"] <= VP[j])].index.values)
            ctl_VP_mat = mat[min(ctl_VP_bin):max(ctl_VP_bin),ctl_VP_values]
            ctl_VP_mat = np.mean(ctl_VP_mat, axis=0)
            ctl_VP_mat = ctl_VP_mat/sum(ctl_VP_mat)  
            
            final_mat = np.column_stack((final_mat,np.flip(ctl_VP_mat)))
    
    #calculate the mean of the control values
    ctl_average = np.mean(final_mat[:,1:len(VP)], axis=1)
    #ctl_average_smooth = np.convolve(ctl_average, kernel, mode='same')
    ctl_average_smooth = scipy.signal.savgol_filter(ctl_average, kernel_size, poly)
    
    #normalize the main VP values on the average of the control values
    with np.errstate(divide='ignore', invalid='ignore'):
        VP_normalized = np.true_divide(final_mat[:,0],ctl_average)
        VP_normalized[VP_normalized == np.inf] = "nan"
        VP_normalized_smooth = np.true_divide(final_mat_smooth,ctl_average_smooth)
        VP_normalized_smooth[VP_normalized_smooth == np.inf] = "nan"
        
    #VP_normalized = np.nan_to_num(VP_normalized)
    final_mat = np.column_stack((final_mat,ctl_average,VP_normalized,final_mat_smooth,ctl_average_smooth,VP_normalized_smooth))

    final_mat = np.row_stack((np.append(chrom,["ctl_average","VP_normalized","VP_smooth"+str(kernel_size),"ctl_average_smooth"+str(kernel_size),"VP_normalized_smooth"+str(kernel_size)]),
                              np.append(VP,["NA","NA","NA","NA","NA"]),
                              np.append(orientation,["NA","NA","NA","NA","NA"]),
                              final_mat))
    np.savetxt(out_dir + str(i) + "_VP_" + str(chrom[0])
               + "_" + str(VP[0]) + "_genome_bin1kb_Condensin_SCN" + str(max_mad) + "_smooth"+ str(kernel_size) + "_poly" + str(poly) + ".tsv", 
               final_mat, delimiter = "\t", fmt='%s')
    print(str(i) + " RE function done")
    

In [11]:
def Function_rDNAleft(frags, mat, chrs, i, out_dir, max_mad, kernel_size, poly):
    #kernel = np.ones(kernel_size)/kernel_size
    VP_bin_size = 3000 #in bp
    chrom = ["chr12", "chr4", "chr14", "chr13", "chr15", "chr11", "chr2"]
    VP = [448000, 747000, 332000, 565000, 624000, 143000, 536000]
    orientation = ["R", "R", "F", "R", "R", "F", "R"]

    #determine the length onto which contacts will be normalized. It is given by the main VP and will be applied to control VPs.
    array_length = int(len(frags.loc[(frags["chrom"]== chrom[0])]) - (VP[0]/1000))
        
    #set the main VP
    main_VP_bin=np.array(frags.loc[(frags["chrom"]== chrom[0]) & (frags["start_pos"] >= VP[0]) & (frags["start_pos"] < (VP[0]+VP_bin_size))].index.values)
    
    #define the stretch of data to look at for the main VP
    if orientation[0] == "F":
        main_VP_values=np.array(frags.loc[(frags["chrom"]== chrom[0]) & (frags["start_pos"] >= VP[0])].index.values)
        main_VP_mat = mat[min(main_VP_bin):max(main_VP_bin),main_VP_values]
        main_VP_mat = np.mean(main_VP_mat, axis=0)
        main_VP_mat = main_VP_mat/sum(main_VP_mat)
        final_mat = main_VP_mat
        #final_mat_smooth = np.convolve(final_mat, kernel, mode='same')
        final_mat_smooth = scipy.signal.savgol_filter(final_mat, kernel_size, poly)
        
    else:
        main_VP_values=np.array(frags.loc[(frags["chrom"]== chrom[0]) & (frags["start_pos"] <= (VP[0]-VP_bin_size))].index.values)
        main_VP_mat = mat[min(main_VP_bin):max(main_VP_bin),main_VP_values]
        main_VP_mat = np.mean(main_VP_mat, axis=0)
        main_VP_mat = main_VP_mat/sum(main_VP_mat)
        final_mat = np.flip(main_VP_mat)
        #final_mat_smooth = np.convolve(final_mat, kernel, mode='same')
        final_mat_smooth = scipy.signal.savgol_filter(final_mat, kernel_size, poly)
        
    distance_bp = len(main_VP_values)*1000
    
    for j in range(1, len(chrom),1):
        ctl_VP_bin=np.array(frags.loc[(frags["chrom"]== chrom[j]) & (frags["start_pos"] >= VP[j]) & (frags["start_pos"] < (VP[j]+VP_bin_size))].index.values)
        #print(chrom[j])
        #print(VP[j])
        #print(orientation[j])
        if orientation[j] == "F":
            ctl_VP_values=np.array(frags.loc[(frags["chrom"]== chrom[j]) & (frags["start_pos"] >= VP[j]) & (frags["start_pos"] < (VP[j]+distance_bp))].index.values)
            ctl_VP_mat = mat[min(ctl_VP_bin):max(ctl_VP_bin),ctl_VP_values]
            ctl_VP_mat = np.mean(ctl_VP_mat, axis=0)
            ctl_VP_mat = ctl_VP_mat/sum(ctl_VP_mat)  
           
            final_mat = np.column_stack((final_mat,ctl_VP_mat))
        else:
            ctl_VP_values=np.array(frags.loc[(frags["chrom"]== chrom[j]) & (frags["start_pos"] > (VP[j]-distance_bp)) & (frags["start_pos"] <= VP[j])].index.values)
            ctl_VP_mat = mat[min(ctl_VP_bin):max(ctl_VP_bin),ctl_VP_values]
            ctl_VP_mat = np.mean(ctl_VP_mat, axis=0)
            ctl_VP_mat = ctl_VP_mat/sum(ctl_VP_mat)  
            
            final_mat = np.column_stack((final_mat,np.flip(ctl_VP_mat)))
    
    #calculate the mean of the control values
    ctl_average = np.mean(final_mat[:,1:len(VP)], axis=1)
    #ctl_average_smooth = np.convolve(ctl_average, kernel, mode='same')
    ctl_average_smooth = scipy.signal.savgol_filter(ctl_average, kernel_size, poly)
    
    #normalize the main VP values on the average of the control values
    with np.errstate(divide='ignore', invalid='ignore'):
        VP_normalized = np.true_divide(final_mat[:,0],ctl_average)
        VP_normalized[VP_normalized == np.inf] = "nan"
        VP_normalized_smooth = np.true_divide(final_mat_smooth,ctl_average_smooth)
        VP_normalized_smooth[VP_normalized_smooth == np.inf] = "nan"
    #VP_normalized = np.nan_to_num(VP_normalized)
    final_mat = np.column_stack((final_mat,ctl_average,VP_normalized,final_mat_smooth,ctl_average_smooth,VP_normalized_smooth))

    final_mat = np.row_stack((np.append(chrom,["ctl_average","VP_normalized","VP_smooth"+str(kernel_size),"ctl_average_smooth"+str(kernel_size),"VP_normalized_smooth"+str(kernel_size)]),
                              np.append(VP,["NA","NA","NA","NA","NA"]),
                              np.append(orientation,["NA","NA","NA","NA","NA"]),
                              final_mat))
    np.savetxt(out_dir + str(i) + "_VP_" + str(chrom[0])
               + "_" + str(VP[0]) + "_rDNA_flankleft448-451_bin1kb_Condensin_SCN" + str(max_mad) + "_smooth"+ str(kernel_size) + "_poly" + str(poly) + ".tsv", 
               final_mat, delimiter = "\t", fmt='%s')    
    print(str(i) + " rDNAleft function done")
    

In [12]:
def Function_RE_chr4_fwd(frags, mat, chrs, i, out_dir, max_mad, kernel_size):
    kernel = np.ones(kernel_size)/kernel_size
    VP_bin_size = 4000 #in bp
    chrom = ["chr4"]
    VP = [678000]
    orientation = ["F"]

    #determine the length onto which contacts will be normalized. It is given by the main VP and will be applied to control VPs.
    array_length = int(len(frags.loc[(frags["chrom"]== chrom[0])]) - (VP[0]/1000))   
   
    #set the main VP
    main_VP_bin=np.array(frags.loc[(frags["chrom"]== chrom[0]) & (frags["start_pos"] >= VP[0]) & (frags["start_pos"] < (VP[0]+VP_bin_size))].index.values)
    
    #define the stretch of data to look at for the main VP
    if orientation[0] == "F":
        main_VP_values=np.array(frags.loc[(frags["chrom"]== chrom[0]) & (frags["start_pos"] >= VP[0]+VP_bin_size)].index.values)
        main_VP_mat = mat[min(main_VP_bin):max(main_VP_bin),main_VP_values]
        main_VP_mat = np.mean(main_VP_mat, axis=0)
        main_VP_mat = main_VP_mat/sum(main_VP_mat)
        final_mat = main_VP_mat
        final_mat_smooth = scipy.signal.savgol_filter(final_mat, kernel_size, 1)
    else:
        main_VP_values=np.array(frags.loc[(frags["chrom"]== chrom[0]) & (frags["start_pos"] <= (VP[0]))].index.values)
        main_VP_mat = mat[min(main_VP_bin):max(main_VP_bin),main_VP_values]
        main_VP_mat = np.mean(main_VP_mat, axis=0)
        main_VP_mat = main_VP_mat/sum(main_VP_mat)
        final_mat = np.flip(main_VP_mat)
        final_mat_smooth = scipy.signal.savgol_filter(final_mat, kernel_size, 1)

    final_mat = np.column_stack((final_mat,final_mat_smooth))
    final_mat = np.row_stack((np.append(chrom,["VP_smooth"+str(kernel_size)]),
                              np.append(VP,["NA"]),
                              np.append(orientation,["NA"]),
                              final_mat))
    np.savetxt(out_dir + str(i) + "_VP_" + str(chrom[0])
               + "_" + str(VP[0]) + "_RE_chr4_678000_FWD_VP4kb_bin1kb_Condensin_SCN" + str(max_mad) + "_smooth" +
               str(kernel_size) + "_poly_" + str(poly) +".tsv", 
               final_mat, delimiter = "\t", fmt='%s')    
    print(str(i) + " RE_chr4 FWD function done")
    

In [13]:
def Function_RE_chr4_rev(frags, mat, chrs, i, out_dir, max_mad, kernel_size):
    kernel = np.ones(kernel_size)/kernel_size
    VP_bin_size = 4000 #in bp
    chrom = ["chr4"]
    VP = [678000]
    orientation = ["R"]

    #determine the length onto which contacts will be normalized. It is given by the main VP and will be applied to control VPs.
    array_length = int(len(frags.loc[(frags["chrom"]== chrom[0])]) - (VP[0]/1000))   
   
    #set the main VP
    main_VP_bin=np.array(frags.loc[(frags["chrom"]== chrom[0]) & (frags["start_pos"] >= VP[0]) & (frags["start_pos"] < (VP[0]+VP_bin_size))].index.values)
    
    #define the stretch of data to look at for the main VP
    if orientation[0] == "F":
        main_VP_values=np.array(frags.loc[(frags["chrom"]== chrom[0]) & (frags["start_pos"] >= VP[0]+VP_bin_size)].index.values)
        main_VP_mat = mat[min(main_VP_bin):max(main_VP_bin),main_VP_values]
        main_VP_mat = np.mean(main_VP_mat, axis=0)
        main_VP_mat = main_VP_mat/sum(main_VP_mat)
        final_mat = main_VP_mat
        final_mat_smooth = scipy.signal.savgol_filter(final_mat, kernel_size, 1)
    else:
        main_VP_values=np.array(frags.loc[(frags["chrom"]== chrom[0]) & (frags["start_pos"] <= (VP[0]))].index.values)
        main_VP_mat = mat[min(main_VP_bin):max(main_VP_bin),main_VP_values]
        main_VP_mat = np.mean(main_VP_mat, axis=0)
        main_VP_mat = main_VP_mat/sum(main_VP_mat)
        final_mat = np.flip(main_VP_mat)
        final_mat_smooth = scipy.signal.savgol_filter(final_mat, kernel_size, 1)

    final_mat = np.column_stack((final_mat,final_mat_smooth))
    final_mat = np.row_stack((np.append(chrom,["VP_smooth"+str(kernel_size)]),
                              np.append(VP,["NA"]),
                              np.append(orientation,["NA"]),
                              final_mat))
    np.savetxt(out_dir + str(i) + "_VP_" + str(chrom[0])
               + "_" + str(VP[0]) + "_RE_chr4_678000_REV_VP4kb_bin1kb_Condensin_SCN" + str(max_mad) + "_smooth"+ str(kernel_size) +
               "_poly_" + str(poly) + ".tsv", 
               final_mat, delimiter = "\t", fmt='%s')    
    print(str(i) + " RE_chr4 REV function done")

Specify the samples

In [7]:
sample=["AP57_S288c_DSB_chr3_rDNA_normal_q20_v312",
"AP103_S288c_DSB_chr3_rDNA_normal_q20_v312",
"AD24_S288c_DSB_chr3_rDNA_cutsite_q20",
"AD161_S288c_DSB_chr3_rDNA_cutsite_q20",
"AD392-400_merged_S288c_DSB_chr3_rDNA_cutsite_q20"]
sample=["AD392-400_merged_S288c_DSB_chr3_rDNA_cutsite_q20"]

Specify the smoothing parameters "kernel_size" and "poly" and run the 4C functions. 
An input cooler file or a graal file together with the fragment list generated by hicstuff can be used.

In [14]:
chrs = pd.read_csv(str(dir_seqs + "Yeast_genome_S288c_R64-2-1_2014/S288c_chr_centro_coordinates.tsv"), sep = "\t")
max_mad=3
kernel_size=39
poly=2
#kernel_size=3
#poly=1
#kernel_size=41 #if using savgol filter of scipy
for i in sample:
    print(str(i) + " starts")
    #Load a fragment file, that gives the correspondance between chromosome and position and the row/column in the dense matrix. 
    #This fragment file is generated by hicstuff using the rebin function. It is generated as part of the master hicstuff script #Script_hicstuff_arima.sh
    
    #frags = pd.read_csv(str(input_dir + "Rebin/S288c/" + str(i) + "_1kb.frags.tsv"), sep = "\t")
    #sparse_mat = hio.load_sparse_matrix(str(input_dir + "Rebin/S288c/" + str(i) +  "_1kb.mat.tsv"))
        
    sparse_mat, frags, chrs = hio.flexible_hic_loader(str(input_dir + "Cool/S288c/" + str(i) +  "_1kb.cool"))
    #sparse_mat, frags, chrs = hio.flexible_hic_loader(str(input_dir + "Cool/S288c/" + str(i) +  "1000_balanced.cool")) #if coming from PSMN
    
    sparse_mat_SCN = hcs.normalize_sparse(sparse_mat, norm='ICE', iterations=40, n_mad=max_mad)   
    #mat = hcv.sparse_to_dense(sparse_mat_SCN)
    

    Function_rDNAleft(frags, mat, chrs, i, out_dir, max_mad, kernel_size, poly)
    Function_RE(frags, mat, chrs, i, out_dir, max_mad, kernel_size, poly)
    Function_RE_chr4_fwd(frags, mat, chrs, i, out_dir, max_mad, kernel_size)
    Function_RE_chr4_rev(frags, mat, chrs, i, out_dir, max_mad, kernel_size)
    print(str(i) + " done")

AD392-400_merged_S288c_DSB_chr3_rDNA_cutsite_q20 starts
AD392-400_merged_S288c_DSB_chr3_rDNA_cutsite_q20 rDNAleft function done
AD392-400_merged_S288c_DSB_chr3_rDNA_cutsite_q20 RE function done
AD392-400_merged_S288c_DSB_chr3_rDNA_cutsite_q20 RE_chr4 FWD function done
AD392-400_merged_S288c_DSB_chr3_rDNA_cutsite_q20 RE_chr4 REV function done
AD392-400_merged_S288c_DSB_chr3_rDNA_cutsite_q20 done
